# import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image, ImageEnhance

# For Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous
from tqdm import tqdm
import time
import os
import random
import pandas as pd
from IPython import display


In [ ]:
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix as cm 
from tensorflow import keras 
from tensorflow.keras.layers import * 
from tensorflow.keras.losses import * 
from tensorflow.keras.models import * 
from tensorflow.keras.metrics import * 
from tensorflow.keras.optimizers import * 
from tensorflow.keras.applications import * 
from tensorflow.keras.preprocessing.image import load_img 
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
#data visualization
import numpy as np
import seaborn as sns
from tqdm import tqdm 
import time 
import os 
import random 
import pandas as pd 
from IPython import display

In [ ]:
data_dir = '/kaggle/input/binary-cropped-crown-of-thorns-dataset/'
unique_labels = os.listdir(data_dir)
all_paths = []
all_labels = []

for label in os.listdir(data_dir):
    for image in os.listdir(data_dir+label):
        all_paths.append(data_dir+label+'/'+image)
        all_labels.append(label)

all_paths, all_labels = shuffle(all_paths, all_labels)
x_train_paths, x_val_paths, y_train, y_val = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), weights='imagenet', include_top=False)

In [ ]:
def augment_image(image):
    if random.uniform(0,1)>0.5:
        image = np.fliplr(image)
    image = Image.fromarray(np.uint8(image))
    image = ImageEnhance.Brightness(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Contrast(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Color(image).enhance(random.uniform(0.6,1.4))
    return image
def open_images(paths, augment=True):
    images = []
    for path in paths:
        image = load_img(path, target_size=(128,128))
        image = np.array(image)
        if augment:
            image = augment_image(image)
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)

def encode_label(labels):
    encoded = []
    for x in labels:
        encoded.append(unique_labels.index(x))
    return np.array(encoded)

def decode_label(labels):
    decoded = []
    for x in labels:
        decoded.append(unique_labels[x])
    return np.array(decoded)


In [ ]:
x=open_images(x_train_paths[0:5000])

In [ ]:
x.shape

In [ ]:
y=encode_label(y_train[0:5000])

In [ ]:
model = Sequential()
model.add(Input(shape=(128,128,3)))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(len(unique_labels),activation='softmax'))
model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#from keras.preprocessing.image import ImageDataGenerator
#x_train = x_train.astype('float32')
#datagen = ImageDataGenerator(zca_whitening=True)
#datagen.fit(x_train)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.000175),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
#steps = int(len(x_train_paths)/batch_size)
#epochs = 6
#batch_size=32
history = model.fit(
    x_train,y_train,
    validation_data=(x_val,y_val),
    batch_size=32,
    epochs=10
    
)

In [ ]:
epochs=10
plt.figure(figsize=(8,8))
plt.grid(True)
plt.plot(history.history['val_sparse_categorical_accuracy'], '.g-', linewidth=2)
plt.plot(history.history['loss'], '.r-', linewidth=2)
plt.title('Model Training History')
plt.xlabel('epoch')
plt.xticks([x for x in range(epochs)])
plt.legend(['Accuracy', 'Loss'], loc='upper left', bbox_to_anchor=(1, 1))
plt.show()